In [58]:
import numpy as np
import pandas as pd
import math
import os.path
import json
import time

In [59]:
DATA_ROOT_PATH = '/src/data/wi-voting'
WI_COUNTY_TURNOUT = 'WI-county.json'
WI_COUNTY_REG_VOTER = 'RegisteredVotersByCounty_11-01-2020.csv'
COUNTY_FISP = 'county_fips_master.csv'

In [60]:
reg_voter_df =  pd.read_csv(os.path.join(DATA_ROOT_PATH, WI_COUNTY_REG_VOTER), encoding='us-ascii')

In [61]:
reg_voter_df['Registered Voters'] = reg_voter_df['Registered Voters'].apply(int)
reg_voter_df=reg_voter_df.rename(columns={'County':'COUNTY'})
reg_voter_df.head(3)

,CountyCode,COUNTY,Registered Voters
0,1,ADAMS COUNTY,12805
1,2,ASHLAND COUNTY,9770
2,3,BARRON COUNTY,27632


In [62]:
reg_voter_df.dtypes

CountyCode            int64
COUNTY               object
Registered Voters     int64
dtype: object

In [63]:
# file -i county_fips_master.csv 
# county_fips_master.csv: application/csv; charset=iso-8859-1

fips_df =  pd.read_csv(os.path.join(DATA_ROOT_PATH, COUNTY_FISP), encoding='iso-8859-1')

In [64]:
fips_df.head(3)

,fips,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name
0,1001,Autauga County,AL,Alabama,Autauga County AL,50.0,3.0,6.0,1.0,1.0,3-6-1-1,South,East South Central
1,1003,Baldwin County,AL,Alabama,Baldwin County AL,50.0,3.0,6.0,1.0,3.0,3-6-1-3,South,East South Central
2,1005,Barbour County,AL,Alabama,Barbour County AL,50.0,3.0,6.0,1.0,5.0,3-6-1-5,South,East South Central


In [65]:
fips_df.dtypes

fips               int64
county_name       object
state_abbr        object
state_name        object
long_name         object
sumlev           float64
region           float64
division         float64
state            float64
county           float64
crosswalk         object
region_name       object
division_name     object
dtype: object

In [66]:
with open(os.path.join(DATA_ROOT_PATH, WI_COUNTY_TURNOUT)) as f:
    wi_county_map = json.load(f)
    turnout_map = wi_county_map['data']
    

In [67]:
print(turnout_map)

{'55001': {'unit': '55001', 'ts': '2020-11-06T00:29:38.960Z', 'pr': 20, 'tp': 20, 'tv': 11806, 'cand': [{'fname': 'Donald', 'name': 'Trump', 'party': 'gop', 'votes': 7362, 'inc': True}, {'fname': 'Joe', 'name': 'Biden', 'party': 'dem', 'votes': 4329, 'winner': 'X'}, {'fname': 'Jo', 'name': 'Jorgensen', 'party': 'lib', 'votes': 85}, {'fname': 'Don', 'name': 'Blankenship', 'party': 'cst', 'votes': 27}, {'fname': 'Brian', 'name': 'Carroll', 'party': 'asp', 'votes': 3}], 'status': 'R'}, '55003': {'unit': '55003', 'ts': '2020-11-06T00:29:38.960Z', 'pr': 17, 'tp': 17, 'tv': 8728, 'cand': [{'fname': 'Joe', 'name': 'Biden', 'party': 'dem', 'votes': 4794, 'winner': 'X'}, {'fname': 'Donald', 'name': 'Trump', 'party': 'gop', 'votes': 3845, 'inc': True}, {'fname': 'Jo', 'name': 'Jorgensen', 'party': 'lib', 'votes': 67}, {'fname': 'Don', 'name': 'Blankenship', 'party': 'cst', 'votes': 11}, {'fname': 'Brian', 'name': 'Carroll', 'party': 'asp', 'votes': 11}], 'status': 'D'}, '55005': {'unit': '55005'

In [68]:
turn_out_per_county = dict()
for county_fips, item in turnout_map.items():
    vote_count = 0
    for cand in item['cand']:
        vote_count = cand['votes'] + vote_count
    if county_fips.isdigit():
        turn_out_per_county[int(county_fips)] = vote_count

In [69]:
# county FISP -> total votes
print(turn_out_per_county)

{55001: 11806, 55003: 8728, 55005: 25316, 55007: 10876, 55009: 143623, 55011: 7798, 55013: 10138, 55015: 30797, 55017: 35908, 55019: 14866, 55021: 33847, 55023: 8695, 55025: 343599, 55027: 48415, 55029: 20076, 55031: 24595, 55033: 23526, 55035: 58069, 55037: 2939, 55039: 57193, 55041: 5050, 55043: 25333, 55045: 21351, 55047: 10644, 55049: 13961, 55051: 4031, 55053: 10179, 55055: 47860, 55057: 13683, 55059: 88467, 55061: 12066, 55063: 67609, 55065: 8547, 55067: 11156, 55069: 16496, 55071: 44477, 55073: 76603, 55075: 22959, 55077: 9050, 55078: 1585, 55079: 457318, 55081: 22560, 55083: 23211, 55085: 24111, 55087: 107956, 55089: 61244, 55091: 4138, 55093: 23260, 55095: 26318, 55097: 40510, 55099: 8642, 55101: 106148, 55103: 9005, 55105: 85150, 55107: 7879, 55109: 56542, 55111: 36172, 55113: 10463, 55115: 22878, 55117: 65855, 55119: 10678, 55121: 15341, 55123: 15907, 55125: 15338, 55127: 57441, 55129: 10353, 55131: 88026, 55133: 267159, 55135: 29081, 55137: 13564, 55139: 93790, 55141: 41196

In [70]:
turn_out_per_county_df = pd.DataFrame(turn_out_per_county.items(), columns=['fips', 'votes'])

In [71]:
turn_out_per_county_df.head(3)

,fips,votes
0,55001,11806
1,55003,8728
2,55005,25316


In [72]:
turn_out_per_county_df = turn_out_per_county_df.merge(fips_df, on='fips', how='left')

In [73]:
turn_out_per_county_df['COUNTY'] = turn_out_per_county_df['county_name'].str.upper()

In [74]:
turn_out_per_county_df.head(3)

,fips,votes,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name,COUNTY
0,55001,11806,Adams County,WI,Wisconsin,Adams County WI,50.0,2.0,3.0,55.0,1.0,2-3-55-1,Midwest,East North Central,ADAMS COUNTY
1,55003,8728,Ashland County,WI,Wisconsin,Ashland County WI,50.0,2.0,3.0,55.0,3.0,2-3-55-3,Midwest,East North Central,ASHLAND COUNTY
2,55005,25316,Barron County,WI,Wisconsin,Barron County WI,50.0,2.0,3.0,55.0,5.0,2-3-55-5,Midwest,East North Central,BARRON COUNTY


In [75]:
turn_out_per_county_df = turn_out_per_county_df.merge(reg_voter_df, on='COUNTY', how='left')

In [76]:
turn_out_per_county_df.head(3)

,fips,votes,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name,COUNTY,CountyCode,Registered Voters
0,55001,11806,Adams County,WI,Wisconsin,Adams County WI,50.0,2.0,3.0,55.0,1.0,2-3-55-1,Midwest,East North Central,ADAMS COUNTY,1,12805
1,55003,8728,Ashland County,WI,Wisconsin,Ashland County WI,50.0,2.0,3.0,55.0,3.0,2-3-55-3,Midwest,East North Central,ASHLAND COUNTY,2,9770
2,55005,25316,Barron County,WI,Wisconsin,Barron County WI,50.0,2.0,3.0,55.0,5.0,2-3-55-5,Midwest,East North Central,BARRON COUNTY,3,27632


In [77]:
turn_out_per_county_df.dtypes

fips                   int64
votes                  int64
county_name           object
state_abbr            object
state_name            object
long_name             object
sumlev               float64
region               float64
division             float64
state                float64
county               float64
crosswalk             object
region_name           object
division_name         object
COUNTY                object
CountyCode             int64
Registered Voters      int64
dtype: object

In [78]:
turn_out_per_county_df['turn_out'] = turn_out_per_county_df['votes'] / turn_out_per_county_df['Registered Voters']

# 2020 Wisconsin Voter Turnout Per County

In [79]:
pd.set_option('display.max_rows', None)
print(turn_out_per_county_df[['COUNTY', 'votes', 'Registered Voters', 'turn_out']].sort_values('turn_out', ascending= False))

                COUNTY   votes  Registered Voters  turn_out
50        PRICE COUNTY    8642               9055  0.954390
60       TAYLOR COUNTY   10678              11294  0.945458
38    MARQUETTE COUNTY    9050               9579  0.944775
28       JUNEAU COUNTY   13683              14567  0.939315
6       BURNETT COUNTY   10138              10795  0.939138
54         RUSK COUNTY    7879               8396  0.938423
69     WAUSHARA COUNTY   13564              14465  0.937712
42       OCONTO COUNTY   23211              24759  0.937477
46        PEPIN COUNTY    4138               4414  0.937472
58      SHAWANO COUNTY   22878              24408  0.937316
30     KEWAUNEE COUNTY   12066              12926  0.933467
7       CALUMET COUNTY   30797              33050  0.931831
66   WASHINGTON COUNTY   88026              94481  0.931679
19  FOND DU LAC COUNTY   57193              61390  0.931634
5       BUFFALO COUNTY    7798               8405  0.927781
32    LAFAYETTE COUNTY    8547          

# 2016 

In [80]:
WI_COUNTY_TURNOUT_2016 = '2016/County by County Report President of the United States Recount.csv'
WI_COUNTY_REG_VOTER_2016 = '2016/registeredvotersbycounty_xlsx_13527.csv'

In [81]:
reg_voter_2016_df = pd.read_csv(os.path.join(DATA_ROOT_PATH, WI_COUNTY_REG_VOTER_2016), encoding='us-ascii')

In [82]:
reg_voter_2016_df.head(3)

,COUNTY,REGISTERED_VOTER
0,ADAMS COUNTY,12476
1,ASHLAND COUNTY,10120
2,BARRON COUNTY,27081


In [83]:
reg_voter_2016_df.dtypes

COUNTY              object
REGISTERED_VOTER     int64
dtype: object

In [84]:
turnout_2016_df = pd.read_csv(os.path.join(DATA_ROOT_PATH, WI_COUNTY_TURNOUT_2016), encoding='us-ascii')

In [85]:
turnout_2016_df.head(3)

,COUNTY,VOTES
0,ADAMS COUNTY,10130
1,ASHLAND COUNTY,8032
2,BARRON COUNTY,22671


In [86]:
turnout_2016_df.dtypes

COUNTY    object
VOTES      int64
dtype: object

In [90]:
turn_out_per_county_2016_df = turnout_2016_df.merge(reg_voter_2016_df, on='COUNTY', how='left')

In [91]:
turn_out_per_county_2016_df['TURN_OUT_2016'] = turn_out_per_county_2016_df['VOTES'] / turn_out_per_county_2016_df['REGISTERED_VOTER']

turn_out_per_county_2016_df=turn_out_per_county_2016_df.rename(columns={'VOTES': 'VOTES_2016', 'REGISTERED_VOTER':'REGISTERED_VOTER_2016'})

# 2016 Wisconsin Voter Turnout Per County

In [93]:
pd.set_option('display.max_rows', None)
print(turn_out_per_county_2016_df[['COUNTY', 'VOTES_2016', 'REGISTERED_VOTER_2016', 'TURN_OUT_2016']].sort_values('TURN_OUT_2016', ascending= False))

                COUNTY  VOTES_2016  REGISTERED_VOTER_2016  TURN_OUT_2016
7       CALUMET COUNTY       26595                  30699       0.866315
54         RUSK COUNTY        7088                   8218       0.862497
8      CHIPPEWA COUNTY       31568                  36651       0.861313
66   WASHINGTON COUNTY       76757                  89277       0.859762
22        GREEN COUNTY       18985                  22117       0.858389
71         WOOD COUNTY       37818                  44072       0.858096
14         DOOR COUNTY       17592                  20527       0.857018
48         POLK COUNTY       22745                  26623       0.854336
19  FOND DU LAC COUNTY       51796                  60664       0.853818
38    MARQUETTE COUNTY        7891                   9256       0.852528
67     WAUKESHA COUNTY      237593                 278896       0.851905
42       OCONTO COUNTY       20206                  23732       0.851424
30     KEWAUNEE COUNTY       10767                 

In [101]:
turn_out_per_county_2020_df = turn_out_per_county_df[['COUNTY', 'votes',  'Registered Voters',  'turn_out']].rename(columns={'votes': 'VOTES_2020',  'Registered Voters': 'REGISTERED_VOTER_2020',  'turn_out': 'TURN_OUT_2020'})

In [103]:
turn_out_per_county_compare_df = turn_out_per_county_2020_df.merge(turn_out_per_county_2016_df, on='COUNTY', how='left')

In [105]:
turn_out_per_county_compare_df['TURN_OUT_DIFF'] = turn_out_per_county_compare_df['TURN_OUT_2020'] - turn_out_per_county_compare_df['TURN_OUT_2016']
turn_out_per_county_compare_df['VOTES_DIFF'] = turn_out_per_county_compare_df['VOTES_2020'] - turn_out_per_county_compare_df['VOTES_2016']
turn_out_per_county_compare_df['REGISTERED_VOTER_DIFF'] = turn_out_per_county_compare_df['REGISTERED_VOTER_2020'] - turn_out_per_county_compare_df['REGISTERED_VOTER_2016']


In [106]:
print(turn_out_per_county_compare_df[['COUNTY', 'TURN_OUT_2016', 'TURN_OUT_2020', 'TURN_OUT_DIFF']].sort_values('TURN_OUT_DIFF', ascending= False))

                COUNTY  TURN_OUT_2016  TURN_OUT_2020  TURN_OUT_DIFF
39    MENOMINEE COUNTY       0.668712       0.885970       0.217258
52     RICHLAND COUNTY       0.777735       0.920851       0.143116
50        PRICE COUNTY       0.814903       0.954390       0.139487
60       TAYLOR COUNTY       0.812055       0.945458       0.133403
11     CRAWFORD COUNTY       0.798182       0.927170       0.128988
25         IRON COUNTY       0.796418       0.913645       0.117227
58      SHAWANO COUNTY       0.820800       0.937316       0.116516
37    MARINETTE COUNTY       0.785960       0.899401       0.113441
0         ADAMS COUNTY       0.811959       0.921984       0.110025
32    LAFAYETTE COUNTY       0.817978       0.927610       0.109632
28       JUNEAU COUNTY       0.830679       0.939315       0.108636
26      JACKSON COUNTY       0.802060       0.903435       0.101375
1       ASHLAND COUNTY       0.793676       0.893347       0.099671
21        GRANT COUNTY       0.815665       0.91